# اوپن اے آئی ماڈلز کی فائن ٹیوننگ

یہ نوٹ بک اوپن اے آئی کی [فائن ٹیوننگ](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) دستاویزات میں فراہم کردہ موجودہ رہنمائی کی بنیاد پر ہے۔

فائن ٹیوننگ آپ کی درخواست کے لیے فاؤنڈیشن ماڈلز کی کارکردگی کو بہتر بناتی ہے، اسے اضافی ڈیٹا اور مخصوص استعمال کے کیس یا منظرنامے سے متعلق سیاق و سباق کے ساتھ دوبارہ تربیت دے کر۔ نوٹ کریں کہ پرامپٹ انجینئرنگ تکنیکیں جیسے _few shot learning_ اور _retrieval augmented generation_ آپ کو معیار کو بہتر بنانے کے لیے متعلقہ ڈیٹا کے ساتھ ڈیفالٹ پرامپٹ کو بڑھانے کی اجازت دیتی ہیں۔ تاہم، یہ طریقے ہدف شدہ فاؤنڈیشن ماڈل کی زیادہ سے زیادہ ٹوکن ونڈو سائز تک محدود ہیں۔

فائن ٹیوننگ کے ذریعے، ہم مؤثر طریقے سے ماڈل کو خود مطلوبہ ڈیٹا کے ساتھ دوبارہ تربیت دے رہے ہیں (جو ہمیں زیادہ مثالیں استعمال کرنے کی اجازت دیتا ہے جو زیادہ سے زیادہ ٹوکن ونڈو میں فٹ نہیں ہو سکتیں) - اور ماڈل کا ایک _حسب ضرورت_ ورژن تعینات کر رہے ہیں جسے انفرنس کے وقت مثالیں فراہم کرنے کی ضرورت نہیں ہوتی۔ یہ نہ صرف ہمارے پرامپٹ ڈیزائن کی مؤثریت کو بہتر بناتا ہے (ہم ٹوکن ونڈو کو دیگر چیزوں کے لیے زیادہ لچک کے ساتھ استعمال کر سکتے ہیں) بلکہ ممکنہ طور پر ہمارے اخراجات کو بھی کم کرتا ہے (انفرنس کے وقت ماڈل کو بھیجے جانے والے ٹوکنز کی تعداد کو کم کر کے)۔

فائن ٹیوننگ کے 4 مراحل ہیں:
1. تربیتی ڈیٹا تیار کریں اور اسے اپ لوڈ کریں۔
1. فائن ٹیونڈ ماڈل حاصل کرنے کے لیے تربیتی کام چلائیں۔
1. فائن ٹیونڈ ماڈل کا جائزہ لیں اور معیار کے لیے دوبارہ کوشش کریں۔
1. جب مطمئن ہوں تو فائن ٹیونڈ ماڈل کو انفرنس کے لیے تعینات کریں۔

نوٹ کریں کہ تمام فاؤنڈیشن ماڈلز فائن ٹیوننگ کی حمایت نہیں کرتے - تازہ ترین معلومات کے لیے [اوپن اے آئی دستاویزات](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) چیک کریں۔ آپ پہلے سے فائن ٹیونڈ ماڈل کو بھی دوبارہ فائن ٹیون کر سکتے ہیں۔ اس ٹیوٹوریل میں، ہم `gpt-35-turbo` کو فائن ٹیوننگ کے لیے اپنے ہدف فاؤنڈیشن ماڈل کے طور پر استعمال کریں گے۔

---


### مرحلہ 1.1: اپنا ڈیٹا سیٹ تیار کریں

آئیے ایک چیٹ بوٹ بنائیں جو آپ کو عناصر کی پیریوڈک ٹیبل کو سمجھنے میں مدد دے جو ایک عنصر کے بارے میں سوالات کے جواب ایک لیمریک کے ذریعے دیتا ہے۔ اس _سادہ_ ٹیوٹوریل میں، ہم صرف ایک ڈیٹا سیٹ بنائیں گے تاکہ ماڈل کو چند نمونہ جوابات کے ساتھ تربیت دی جا سکے جو ڈیٹا کے متوقع فارمیٹ کو ظاہر کرتے ہیں۔ حقیقی دنیا کے استعمال کے معاملے میں، آپ کو بہت زیادہ مثالوں کے ساتھ ایک ڈیٹا سیٹ تیار کرنے کی ضرورت ہوگی۔ آپ ایک کھلا ڈیٹا سیٹ بھی استعمال کر سکتے ہیں (اپنے درخواست کے دائرہ کار کے لیے) اگر کوئی موجود ہو، اور اسے فائن ٹیوننگ کے لیے دوبارہ فارمیٹ کر سکتے ہیں۔

چونکہ ہم `gpt-35-turbo` پر توجہ مرکوز کر رہے ہیں اور ایک سنگل ٹرن جواب (چیٹ کمپلیشن) کی تلاش میں ہیں، ہم [اس تجویز کردہ فارمیٹ](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) کا استعمال کرتے ہوئے مثالیں بنا سکتے ہیں جو OpenAI چیٹ کمپلیشن کی ضروریات کی عکاسی کرتا ہے۔ اگر آپ کثیر ٹرن بات چیت کا مواد توقع کرتے ہیں، تو آپ [کثیر ٹرن مثال فارمیٹ](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) استعمال کریں گے جس میں `weight` پیرامیٹر شامل ہوتا ہے تاکہ یہ اشارہ کیا جا سکے کہ کون سے پیغامات فائن ٹیوننگ کے عمل میں استعمال ہونے چاہئیں (یا نہیں)۔

ہم یہاں اپنے ٹیوٹوریل کے لیے آسان سنگل ٹرن فارمیٹ استعمال کریں گے۔ ڈیٹا [jsonl فارمیٹ](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) میں ہے جس میں ہر لائن پر 1 ریکارڈ ہوتا ہے، ہر ایک JSON فارمیٹڈ آبجیکٹ کی صورت میں پیش کیا جاتا ہے۔ نیچے دیا گیا ٹکڑا 2 ریکارڈز کو بطور نمونہ دکھاتا ہے - مکمل نمونہ سیٹ (10 مثالیں) کے لیے [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) دیکھیں جو ہم اپنے فائن ٹیوننگ ٹیوٹوریل کے لیے استعمال کریں گے۔ **نوٹ:** ہر ریکارڈ _ضرور_ ایک ہی لائن میں تعریف کیا جانا چاہیے (فارمیٹ شدہ JSON فائل کی طرح لائنوں میں تقسیم نہیں کیا جانا چاہیے)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

حقیقی دنیا کے استعمال کے معاملے میں آپ کو اچھے نتائج کے لیے بہت زیادہ مثالوں کا سیٹ درکار ہوگا - اس کا توازن جوابات کے معیار اور فائن ٹیوننگ کے وقت/لاگت کے درمیان ہوگا۔ ہم ایک چھوٹا سیٹ استعمال کر رہے ہیں تاکہ ہم فائن ٹیوننگ کو جلد مکمل کر سکیں اور عمل کی وضاحت کر سکیں۔ مزید پیچیدہ فائن ٹیوننگ ٹیوٹوریل کے لیے [اس OpenAI کک بک کی مثال](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) دیکھیں۔


---

### مرحلہ 1.2 اپنا ڈیٹا سیٹ اپ لوڈ کریں

ڈیٹا کو فائلز API کا استعمال کرتے ہوئے اپ لوڈ کریں [جیسا کہ یہاں بیان کیا گیا ہے](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file)۔ نوٹ کریں کہ اس کوڈ کو چلانے کے لیے، آپ کو پہلے درج ذیل مراحل مکمل کرنے ہوں گے:
 - `openai` پائتھن پیکیج انسٹال کیا ہوا ہو (یقینی بنائیں کہ آپ ورژن >=0.28.0 استعمال کر رہے ہیں تاکہ تازہ ترین خصوصیات دستیاب ہوں)
 - `OPENAI_API_KEY` ماحول کی متغیر کو اپنے OpenAI API کلید پر سیٹ کیا ہوا ہو
مزید جاننے کے لیے، کورس کے لیے فراہم کردہ [سیٹ اپ گائیڈ](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) دیکھیں۔

اب، اپنے مقامی JSONL فائل سے اپ لوڈ کے لیے فائل بنانے کے لیے کوڈ چلائیں۔


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### مرحلہ 2.1: SDK کے ساتھ Fine-tuning جاب بنائیں


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### مرحلہ 2.2: کام کی حیثیت چیک کریں

یہاں کچھ چیزیں ہیں جو آپ `client.fine_tuning.jobs` API کے ساتھ کر سکتے ہیں:
- `client.fine_tuning.jobs.list(limit=<n>)` - آخری n فائن ٹیوننگ کاموں کی فہرست بنائیں
- `client.fine_tuning.jobs.retrieve(<job_id>)` - کسی مخصوص فائن ٹیوننگ کام کی تفصیلات حاصل کریں
- `client.fine_tuning.jobs.cancel(<job_id>)` - فائن ٹیوننگ کام منسوخ کریں
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - کام سے n تک واقعات کی فہرست بنائیں
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

عمل کا پہلا مرحلہ _تربیتی فائل کی تصدیق_ ہے تاکہ یہ یقینی بنایا جا سکے کہ ڈیٹا صحیح فارمیٹ میں ہے۔


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### مرحلہ 2.3: پیش رفت کی نگرانی کے لیے واقعات کو ٹریک کریں


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### مرحلہ 2.4: اوپن اے آئی ڈیش بورڈ میں حیثیت دیکھیں


آپ اوپن اے آئی کی ویب سائٹ پر جا کر اور پلیٹ فارم کے _Fine-tuning_ سیکشن کو دریافت کر کے بھی اسٹیٹس دیکھ سکتے ہیں۔ یہ آپ کو موجودہ کام کی حالت دکھائے گا، اور آپ کو پچھلے کام کی انجام دہی کی تاریخ کو بھی ٹریک کرنے دے گا۔ اس اسکرین شاٹ میں، آپ دیکھ سکتے ہیں کہ پچھلی انجام دہی ناکام ہوئی، اور دوسری بار چلانے میں کامیابی حاصل ہوئی۔ سیاق و سباق کے لیے، یہ اس وقت ہوا جب پہلی بار چلانے میں JSON فائل میں غلط فارمیٹ کیے گئے ریکارڈز استعمال ہوئے تھے - ایک بار درست ہونے کے بعد، دوسری بار چلانا کامیابی سے مکمل ہوا اور ماڈل کو استعمال کے لیے دستیاب کر دیا گیا۔ 

![Fine-tuning job status](../../../../../translated_images/ur/fine-tuned-model-status.563271727bf7bfba.webp)


آپ بصری ڈیش بورڈ میں نیچے مزید سکرول کر کے اسٹیٹس پیغامات اور میٹرکس بھی دیکھ سکتے ہیں جیسا کہ دکھایا گیا ہے:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/ur/fine-tuned-messages-panel.4ed0c2da5ea1313b.webp) |  ![Metrics](../../../../../translated_images/ur/fine-tuned-metrics-panel.700d7e4995a65229.webp)|


---

### مرحلہ 3.1: کوڈ میں ID حاصل کریں اور فائن ٹیونڈ ماڈل کا ٹیسٹ کریں


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### مرحلہ 3.2: فائن ٹونڈ ماڈل کو پلے گراؤنڈ میں لوڈ اور ٹیسٹ کریں

اب آپ فائن ٹونڈ ماڈل کو دو طریقوں سے ٹیسٹ کر سکتے ہیں۔ سب سے پہلے، آپ پلے گراؤنڈ پر جا کر ماڈلز ڈراپ ڈاؤن سے اپنی نئی فائن ٹونڈ ماڈل کو منتخب کر سکتے ہیں جو اختیارات میں شامل ہے۔ دوسرا طریقہ "Playground" آپشن استعمال کرنا ہے جو فائن ٹوننگ پینل میں دکھایا گیا ہے (اوپر اسکرین شاٹ دیکھیں) جو درج ذیل _موازنہ_ ویو کو لانچ کرتا ہے جو بنیاد اور فائن ٹونڈ ماڈل ورژنز کو ایک ساتھ تیز جائزے کے لیے دکھاتا ہے۔

![Fine-tuning job status](../../../../../translated_images/ur/fine-tuned-playground-compare.56e06f0ad8922016.webp)

بس اپنے تربیتی ڈیٹا میں استعمال ہونے والے سسٹم کانٹیکسٹ کو بھر دیں اور اپنا ٹیسٹ سوال فراہم کریں۔ آپ دیکھیں گے کہ دونوں طرف ایک ہی کانٹیکسٹ اور سوال اپ ڈیٹ ہو جاتے ہیں۔ موازنہ چلائیں اور آپ ان کے آؤٹ پٹ میں فرق دیکھیں گے۔ _نوٹ کریں کہ فائن ٹونڈ ماڈل آپ کی مثالوں میں فراہم کردہ فارمیٹ میں جواب دیتا ہے جبکہ بنیاد ماڈل صرف سسٹم پرامپٹ کی پیروی کرتا ہے_۔

![Fine-tuning job status](../../../../../translated_images/ur/fine-tuned-playground-launch.5a26495c983c6350.webp)

آپ دیکھیں گے کہ موازنہ ہر ماڈل کے لیے ٹوکن کی تعداد اور انفرنس کے لیے لگنے والا وقت بھی فراہم کرتا ہے۔ **یہ مخصوص مثال ایک سادہ سی ہے جو عمل دکھانے کے لیے ہے لیکن حقیقی دنیا کے ڈیٹا سیٹ یا منظر نامے کی عکاسی نہیں کرتی**۔ آپ دیکھ سکتے ہیں کہ دونوں نمونوں میں ٹوکن کی تعداد ایک جیسی ہے (سسٹم کانٹیکسٹ اور یوزر پرامپٹ ایک جیسے ہیں) جبکہ فائن ٹونڈ ماڈل انفرنس کے لیے زیادہ وقت لیتا ہے (کسٹم ماڈل)۔

حقیقی دنیا کے منظر ناموں میں، آپ اس طرح کی سادہ مثال استعمال نہیں کریں گے، بلکہ حقیقی ڈیٹا کے خلاف فائن ٹوننگ کریں گے (مثلاً، کسٹمر سروس کے لیے پروڈکٹ کیٹلاگ) جہاں جواب کی معیار زیادہ واضح ہوگی۔ _اس سیاق و سباق میں، بنیاد ماڈل کے ساتھ مساوی جواب کی معیار حاصل کرنے کے لیے زیادہ کسٹم پرامپٹ انجینئرنگ کی ضرورت ہوگی جو ٹوکن کے استعمال کو بڑھائے گی اور ممکنہ طور پر انفرنس کے لیے متعلقہ پروسیسنگ وقت کو بھی بڑھائے گی۔_ _اسے آزمانے کے لیے، OpenAI Cookbook میں فائن ٹوننگ کی مثالیں دیکھیں اور شروع کریں۔_

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**دستخطی نوٹ**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کے ذریعے ترجمہ کی گئی ہے۔ اگرچہ ہم درستگی کے لیے کوشاں ہیں، براہ کرم اس بات سے آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا عدم درستیاں ہو سکتی ہیں۔ اصل دستاویز اپنی مادری زبان میں ہی معتبر ماخذ سمجھی جانی چاہیے۔ اہم معلومات کے لیے پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کی ذمہ داری ہم پر عائد نہیں ہوتی۔
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
